In [1]:
import pandas as pd
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [2]:
train = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/train.csv')
test = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/test.csv')
movies = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/movies.csv', delimiter= '::', encoding= 'latin1')
users = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/users.csv', delimiter='::')

/var/folders/5j/7vfy0ndj133gm55cj1czw8rr0000gn/T/ipykernel_63659/2668559298.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/movies.csv', delimiter= '::', encoding= 'latin1')
/var/folders/5j/7vfy0ndj133gm55cj1czw8rr0000gn/T/ipykernel_63659/2668559298.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users = pd.read_csv('/Users/Stefano_1/Documents/Data/ml2017-fall-hw5/users.csv', delimiter='::')


In [3]:
movies['year'] =  movies['Title'].str.extract(r'\((\d{4})\)', expand=False)
movies['Title'] = movies['Title'].str.replace(r'\(\d{4}\)$',"")
movies['Genres'] = movies['Genres'].apply(lambda x: x.split('|'))
movies.head()

/var/folders/5j/7vfy0ndj133gm55cj1czw8rr0000gn/T/ipykernel_63659/1660701533.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  movies['Title'] = movies['Title'].str.replace(r'\(\d{4}\)$',"")


,movieID,Title,Genres,year
0,1,Toy Story,"[Animation, Children's, Comedy]",1995
1,2,Jumanji,"[Adventure, Children's, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [4]:
train

,TrainDataID,UserID,MovieID,Rating
0,1,796,1193,5
1,2,796,661,3
2,3,796,914,3
3,4,796,3408,4
4,5,796,2355,5
...,...,...,...,...
899868,899869,2683,1090,3
899869,899870,2683,1091,1
899870,899871,2683,1094,5
899871,899872,2683,562,5


In [5]:
train_full = train.merge(users, on = 'UserID').merge(movies, left_on = 'MovieID', right_on='movieID')
train_full = train_full.drop(['movieID', 'Title'], axis = 1)

In [ ]:
test_full = test.merge(users, on = 'UserID').merge(movies, left_on = 'MovieID', right_on='movieID')
test_full = test.drop(['movieID', 'Title'], axis = 1)

In [6]:
reader = Reader(rating_scale= (1,5))
data = Dataset.load_from_df(train_full[['UserID', 'MovieID', 'Rating']], reader)

In [7]:
data

In [8]:
param_grid = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 30, 40],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.05, 0.1]
}

In [9]:
#svd = SVD()
gs = GridSearchCV(algo_class=SVD, param_grid=param_grid, measures=['rmse'], n_jobs=-1)
gs.fit(data)

In [10]:
gs.best_score

best_params = gs.best_params['rmse']

svd = SVD(**best_params)

In [11]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [13]:
predictions = []

for _, row in train_full.iterrows():
    user = row['UserID']
    movie = row['MovieID']

    res = svd.predict(user, movie)

    predictions.append(res.est)


train_full['svd_pred'] = predictions
#submission = pd.DataFrame(predictions, columns=['TestDataID', 'Rating'])

#submission['Rating'] = submission['Rating'].round()


In [44]:
submission.to_csv('/Users/Stefano_1/Documents/Data/Kaggle Movie Rec/sub1_svd_cv_RF.csv', index_label= 'id', index = False)